## Association Rules - BAKERY 
Use dataset **75000** (select suitable files in this folder) to build the model to **identify sets of items** that are frequently bought together with two cases: 
- Case 1: Using: **Id**
- Case 2: Using: **Flavor and Food name (goods.csv)**.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [3]:
# Pre-processing data
from pyspark.sql.functions import collect_list, col, count, collect_set
# Convert list array to string
from pyspark.sql.types import StringType

In [4]:
spark = SparkSession.builder.appName('bakery').getOrCreate()

### Case 1: Using Product ID

In [5]:
data = spark.read.csv("Data/75000/75000i.csv", header=False, inferSchema=True)

In [6]:
data.count()

266209

In [7]:
data.show(10)

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
|  1|  1| 21|
|  1|  5| 11|
|  2|  1|  7|
|  2|  3| 11|
|  2|  4| 37|
|  2|  3| 45|
|  3|  5|  3|
|  3|  3| 42|
|  3|  3| 33|
|  4|  1|  5|
+---+---+---+
only showing top 10 rows



In [8]:
data = data.withColumnRenamed('_c0', 'order_id')\
            .withColumnRenamed('_c1', 'items')\
            .withColumnRenamed('_c2', 'product_id')

In [9]:
data.show(5)

+--------+-----+----------+
|order_id|items|product_id|
+--------+-----+----------+
|       1|    1|        21|
|       1|    5|        11|
|       2|    1|         7|
|       2|    3|        11|
|       2|    4|        37|
+--------+-----+----------+
only showing top 5 rows



In [10]:
data.createOrReplaceTempView("order_products_train")

In [11]:
products = spark.sql("select distinct product_id from order_products_train")
products.count()

50

In [12]:
rawData = spark.sql("select * from order_products_train")
# data
baskets = rawData.groupBy('order_id').agg(collect_set('product_id').alias('items'))

In [13]:
baskets.show(5, truncate=False)

+--------+---------------+
|order_id|items          |
+--------+---------------+
|1       |[21, 11]       |
|2       |[45, 37, 7, 11]|
|3       |[33, 42, 3]    |
|4       |[12, 5, 17, 47]|
|5       |[42, 6, 18]    |
+--------+---------------+
only showing top 5 rows



In [14]:
baskets.count()

75000

In [15]:
from pyspark.ml.fpm import FPGrowth

In [16]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model = fpGrowth.fit(baskets)

In [17]:
# Display frequent itemsets.
model.freqItemsets.show()

+--------+----+
|   items|freq|
+--------+----+
|    [19]|5685|
|[19, 27]| 359|
|[19, 33]| 334|
| [19, 1]|2764|
|[19, 28]| 408|
|[19, 37]| 274|
|[19, 35]| 312|
|[19, 16]| 286|
| [19, 4]| 388|
|[19, 46]| 324|
|[19, 15]| 298|
| [19, 5]| 323|
|[19, 22]| 368|
|[19, 32]| 297|
|[19, 45]| 344|
|[19, 47]| 331|
| [19, 3]| 294|
|[19, 14]| 350|
|[19, 11]| 296|
| [19, 0]| 305|
+--------+----+
only showing top 20 rows



In [18]:
# transform 
mostPopularItemInABasket = model.transform(baskets)

In [58]:
mostPopularItemInABasket.show(3, truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 order_id   | 1                                                                                                                                                                                   
 items      | [21, 11]                                                                                                                                                                            
 prediction | [19, 33, 27, 1, 28, 35, 16, 46, 4, 15, 22, 5, 32, 45, 47, 14, 0, 18, 49, 9, 44, 42, 7, 17, 29, 2, 40, 37, 3]                                                                        
-RECORD 1-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 order_id   | 2          

In [20]:
mostPopularItemInABasket.write.parquet("MPIB.parquet", mode = "overwrite")

### Case 1: Using ID

### Case 2: Using - Flavor and Food name

In [30]:
product_data = spark.read.csv('Data/75000/goods.csv', header=True, 
                              inferSchema=True)

In [31]:
product_data.show(5)

+---+------------+------+-----+------+
| Id|      Flavor|  Food|Price|  Type|
+---+------------+------+-----+------+
|  0| 'Chocolate'|'Cake'| 8.95|'Food'|
|  1|     'Lemon'|'Cake'| 8.95|'Food'|
|  2|    'Casino'|'Cake'|15.95|'Food'|
|  3|     'Opera'|'Cake'|15.95|'Food'|
|  4|'Strawberry'|'Cake'|11.95|'Food'|
+---+------------+------+-----+------+
only showing top 5 rows



In [39]:
product_data.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Flavor: string (nullable = true)
 |-- Food: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Type: string (nullable = true)



In [37]:
from pyspark.sql.functions import regexp_replace

In [42]:
# Loại bỏ dấu ' trước và sau chữ trong các cột Flavor, Food và Type
product_data = product_data.withColumn("Flavor", regexp_replace("Flavor", "'", ""))\
            .withColumn("Food", regexp_replace("Food", "'", ""))\
            .withColumn("Type", regexp_replace("Type", "'", ""))

In [43]:
product_data.createOrReplaceTempView("products")

In [47]:
query = '''select concat(p.Flavor,' ',p.Food) as product_name, o.order_id 
            from products p inner join order_products_train o 
            where o.product_id = p.Id'''
rawData_1 = spark.sql(query)
baskets_1 = rawData_1.groupBy("order_id").agg(collect_set("product_name").alias("items"))

In [49]:
baskets_1.show(3, truncate=False)

+--------+-------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                  |
+--------+-------------------------------------------------------------------------------------------------------+
|1       |[Ganache Cookie, Apple Pie]                                                                            |
|6       |[Strawberry Cake, Chocolate Croissant, Casino Cake]                                                    |
|12      |[Almond Twist, Ganache Cookie, Opera Cake, Single Espresso, Casino Cake, Raspberry Lemonade, Apple Pie]|
+--------+-------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [51]:
model_1 = fpGrowth.fit(baskets_1)

In [52]:
# Display frequent itemsets
model_1.freqItemsets.show(truncate=False)

+---------------------------------------+----+
|items                                  |freq|
+---------------------------------------+----+
|[Vanilla Meringue]                     |3179|
|[Vanilla Meringue, Lemon Tart]         |252 |
|[Vanilla Meringue, Marzipan Cookie]    |277 |
|[Vanilla Meringue, Cheese Croissant]   |260 |
|[Vanilla Meringue, Chocolate Tart]     |233 |
|[Vanilla Meringue, Lemon Cake]         |293 |
|[Vanilla Meringue, Tuile Cookie]       |312 |
|[Vanilla Meringue, Apricot Danish]     |249 |
|[Vanilla Meringue, Blueberry Tart]     |229 |
|[Vanilla Meringue, Chocolate Coffee]   |245 |
|[Vanilla Meringue, Strawberry Cake]    |299 |
|[Vanilla Meringue, Blackberry Tart]    |229 |
|[Vanilla Meringue, Gongolais Cookie]   |293 |
|[Vanilla Meringue, Truffle Cake]       |255 |
|[Vanilla Meringue, Apricot Croissant]  |229 |
|[Vanilla Meringue, Hot Coffee]         |227 |
|[Vanilla Meringue, Vanilla Frappuccino]|260 |
|[Vanilla Meringue, Berry Tart]         |281 |
|[Vanilla Mer

In [54]:
mostPopularItemInABasket_1 = model_1.transform(baskets_1)

In [57]:
mostPopularItemInABasket_1.show(3, truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 order_id   | 1                                                                                                                                                                                                                                                    